In [1]:
import tifffile
import matplotlib.pyplot as plt
import numpy as np
from csbdeep.data import RawData, create_patches, permute_axes, Transform
import json

In [2]:
f = open("sample_data_config.json",'rb')
config = json.load(f)
source_dirs = os.listdir(os.path.join(config["basepath"],config["source_dir"]))
print(source_dirs)
source_dirs = [os.path.join(config["source_dir"],s) for s in source_dirs]

raw_data = RawData.from_folder (
        basepath    = config["basepath"],
        source_dirs = source_dirs,
        target_dir  = config["target_dir"],
        axes        = config["axes"],
    )

['t00', 't01', 't02', 't03', 't04', 't05', 't06', 't07', 't08', 't09', 't10', 't11']


In [3]:
def median_filter(threshold=0.4, percentile=99.9):


    from scipy.ndimage.filters import maximum_filter, median_filter
    def _filter(datas, patch_size, dtype=np.float32):
        image = datas[0]
        if dtype is not None:
            image = image.astype(dtype)
        # make max filter patch_size smaller to avoid only few non-bg pixel close to image border
        patch_size = [(p//2 if p>1 else p) for p in patch_size]
        filtered = median_filter(image, patch_size, mode='constant')
        #print(filtered)
        return filtered > threshold * np.percentile(image,percentile)
    return _filter

X, Y, XY_axes = create_patches (
            raw_data            = raw_data,
            patch_size          = (2,64,64),
            n_patches_per_image = config["n_patches_per_image"],
            #transforms          = transforms,
            save_file           = config["save_file"],
            patch_filter= median_filter
        )


  144 raw images x    1 transformations   =   144 images
  144 images     x  512 patches per image = 73728 patches in total
Input data:
/media/panagiotis/TOSHIBA EXT/Research/Denoising/Data/GFPNegativeCells_20191114/raw_tifs/: target='860_avg', sources=['755/t00', '755/t01', '755/t02', '755/t03', '755/t04', '755/t05', '755/t06', '755/t07', '755/t08', '755/t09', '755/t10', '755/t11'], axes='CYX', pattern='*.tif*'
Transformations:
1 x Identity
Patch size:
2 x 64 x 64
  0%|          | 0/144 [00:00<?, ?it/s]


ValueError: 'patch_filter' didn't return any region to sample from